# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [6]:
RANDOM_SEED = 0

In [7]:
df = pd.read_csv('.//datasets/5ds_tariff_recommendation.csv')

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Видим, что разрядность слишком высокая, оптимизируем типы данных.

In [10]:
float_features = df.select_dtypes(include='float64').columns
df[float_features] = df[float_features].apply(pd.to_numeric, downcast='float')
df['is_ultra'] = pd.to_numeric(df['is_ultra'], downcast='signed')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float32
 1   minutes   3214 non-null   float32
 2   messages  3214 non-null   float32
 3   mb_used   3214 non-null   float32
 4   is_ultra  3214 non-null   int8   
dtypes: float32(4), int8(1)
memory usage: 53.5 KB


In [12]:
df.sample(5, random_state=RANDOM_SEED)

,calls,minutes,messages,mb_used,is_ultra
2568,82.0,580.090027,96.0,21463.800781,0
2200,29.0,217.149994,46.0,16933.720703,0
306,36.0,250.880005,49.0,18965.019531,0
831,133.0,972.700012,83.0,38552.621094,0
2781,25.0,155.550003,98.0,17856.189453,0


In [13]:
pd.DataFrame(round((df.isna().mean()*100), 2)).style.background_gradient('coolwarm')\
                                                                .set_caption('% пропусков').format({0:'{:,.2%}'.format})

,0
calls,0.00%
minutes,0.00%
messages,0.00%
mb_used,0.00%
is_ultra,0.00%


### Выводы

- Данные состоят из 3214 строк и 5 столбцов: calls, minutes, messages, mb_used, is_ultra.
- Пропусков нет
- Данные по тарифам бьются в соотношении 70%/30%
- Поскольку разрядность в данных высокая, мы оптимизировали типы данных.

## Разбейте данные на выборки

Извлечем признаки и целевой признак.

In [14]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [15]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, 
                                                                              random_state=RANDOM_SEED)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5,
                                                                           random_state=RANDOM_SEED)

***Проверка того, что выборки сформированы корректно.***

In [24]:
print('Train: ', features_train.shape, target_train.shape)
print('----------------')
print('Train: ', features_valid.shape, target_valid.shape)
print('----------------')
print('Train: ', features_test.shape, target_test.shape)


Train:  (1928, 4) (1928,)
----------------
Train:  (643, 4) (643,)
----------------
Train:  (643, 4) (643,)


## Исследуйте модели

Обучим модель дерева решений и выберем лучшую по метрике accuracy.

In [17]:
best_model_dec_tree = None
best_result_dec_tree = 0
depth = 0
for depth in range(1, 10):
    model_dec_tree = DecisionTreeClassifier(random_state=RANDOM_SEED, max_depth = depth) #обучение с заданной глубиной дерева
    model_dec_tree.fit(features_train, target_train) #обучение модели
    predictions_valid = model_dec_tree.predict(features_valid) #Предсказания
    result_dec_tree = accuracy_score(target_valid, predictions_valid) #качество модели
    if result_dec_tree > best_result_dec_tree:
        best_model_dec_tree = model_dec_tree
        best_result_dec_tree = result_dec_tree
        max_depth  = depth
        
print('Accuracy лучшей модели: ', best_result_dec_tree, ", max_depth = ", max_depth)

Accuracy лучшей модели:  0.8118195956454122 , max_depth =  7


Лучше всего по accuracy показала себя модель дерева решений с максимальной глубиной max_depth = 7 : 0.8118195956454122.

Обучим модель случайного леса и выберем наиболее подходящие гиперпараметры для нашей модели исходя из Accuracy.

In [18]:
best_model_rand_for = None
best_result_rand_for = 0
n_estimators = 0
for est in range(1, 20):
    model_rand_for = RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=est)
    model_rand_for.fit(features_train, target_train)
    result_rand_for = model_rand_for.score(features_valid, target_valid)
    if result_rand_for > best_result_rand_for:
        best_model_rand_for = model_rand_for #наилучшая модель
        best_result_rand_for = result_rand_for #значение accuraccy на валидационной выборке
        n_estimators = est #количество деревьев
print('Accuracy лучшей модели: {:.2f}, n_estimators = {:.0f}'.format(best_result_rand_for, n_estimators))

Accuracy лучшей модели: 0.79, n_estimators = 10


Лучше всего по accuracy показала себя модель случайного леса с количеством деревьев = 10 .

Обучим модель логистической регрессии.

In [19]:
model_log_reg = LogisticRegression(random_state=RANDOM_SEED)
model_log_reg.fit(features_train, target_train)
result = model_log_reg.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке: ", result)

Accuracy модели логистической регрессии на валидационной выборке:  0.7433903576982893


Accuracy модели логистической регрессии на валидационной выборке:  0.71. Она уступает по качеству модели случайного леса и дерева решений.

## Проверьте модель на тестовой выборке

In [20]:
predictions_test_dec_tree = model_dec_tree.predict(features_test) #Предсказания
result_dec_tree = accuracy_score(target_test, predictions_test_dec_tree) #качество модели
print("Accuracy на тестовой выборке: {:.10f}".format(result_dec_tree))

Accuracy на тестовой выборке: 0.7620528771


На тестовой выборке модель дерева решений показала себя хуже, чем на валидационной. Accuracy = 0.762

In [21]:
predictions_rand_for = model_rand_for.predict(features_test)
result_rand_for = accuracy_score(target_test, predictions_rand_for)
print("Accuracy на тестовой выборке: {:.10f}".format(result_rand_for))

Accuracy на тестовой выборке: 0.7791601866


Модель случайного леса с количеством деревьев 10 показала себя чуть хуже, чем на валидационной выборке, accuracy равно 0.779, но в то же время лучше, чем дерево решений.

In [22]:
predictions_log_reg = model_log_reg.predict(features_test)
result_log_reg = accuracy_score(target_test, predictions_log_reg)

print("Accuracy модели логистической регрессии на валидационной выборке: {:.2f}".format(result_log_reg))

Accuracy модели логистической регрессии на валидационной выборке: 0.75


Модель логистической регрессии нам не подходит. Accuracy равно 0.7.

## (бонус) Проверьте модели на адекватность

Сравним получившуюся модель с константной.

In [25]:
df['is_ultra'].value_counts()/df.shape[0]

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
